In [9]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone

In [2]:
DOT_ENV_PATH = find_dotenv()
load_dotenv(DOT_ENV_PATH)

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
PINECONE_INDEX_NAME = "equity-sense-vdb"
LANGCHAIN_PROJECT="defaultv2"

In [4]:
# Instantiate LLM Model
llm = ChatOpenAI()

In [5]:
# Declare PineCone Client
pc = Pinecone(api_key=PINECONE_API_KEY)

In [6]:
# Get Index Object
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize embedding model
embeddings = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY, 
    model='text-embedding-ada-002'
)

# Create VectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [7]:
retriever = vector_store.as_retriever()

In [10]:
parser = StrOutputParser()

In [11]:
system_prompt = (
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise. \n\n
    Context: {context}"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Which year does NVIDIA has the highest Net Income?"})
response["answer"]

'NVIDIA had the highest Net Income in the year 2024 with 29.76 Billion USD.'

In [12]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [14]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is NVIDIA Net Income?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

print(ai_msg_1["answer"] + "\n\n")

second_question = "Order these income from highest to lowest?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

The NVIDIA Net Income for 2024-01-31 was 29.76 Billion USD, for 2023-01-31 was 4.368 Billion USD, for 2022-01-31 was 9.752 Billion USD, and for 2021-01-31 was 4.332 Billion USD. The net income values represent the profit generated by NVIDIA after accounting for all expenses and taxes.


The NVIDIA Net Income values from highest to lowest are: 29.76 Billion USD for 2024-01-31, 9.752 Billion USD for 2022-01-31, 4.368 Billion USD for 2023-01-31, and 4.332 Billion USD for 2021-01-31.


In [16]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# We define a dict representing the state of the application.
# This state has the same input and output keys as `rag_chain`.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


# We then define a simple node that runs the `rag_chain`.
# The `return` values of the node update the graph state, so here we just
# update the chat history with the input message and response.
def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


# Our graph consists only of one node:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile the graph with a checkpointer object.
# This persists the state, in this case in memory.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [17]:
config = {"configurable": {"thread_id": "abc124"}}

result = app.invoke(
    {"input": "What is NVIDIA Net Income?"},
    config=config,
)
print(result["answer"])

NVIDIA's Net Income for the date 2024-01-31 was 29.76 Billion USD, for 2023-01-31 was 4.368 Billion USD, for 2022-01-31 was 9.752 Billion USD, and for 2021-01-31 was 4.332 Billion USD. The Net Income represents the company's total earnings after expenses such as taxes and operating costs have been deducted.


In [18]:
result = app.invoke(
    {"input": "Order NVIDIA income from highest to lowest?"},
    config=config,
)
print(result["answer"])

The order of NVIDIA's Net Income from highest to lowest is as follows: 
1. 2024-01-31: 29.76 Billion USD
2. 2022-01-31: 9.752 Billion USD
3. 2023-01-31: 4.368 Billion USD
4. 2021-01-31: 4.332 Billion USD.


In [19]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

What is NVIDIA Net Income?
================================== Ai Message ==================================

NVIDIA's Net Income for the date 2024-01-31 was 29.76 Billion USD, for 2023-01-31 was 4.368 Billion USD, for 2022-01-31 was 9.752 Billion USD, and for 2021-01-31 was 4.332 Billion USD. The Net Income represents the company's total earnings after expenses such as taxes and operating costs have been deducted.
================================ Human Message =================================

Order NVIDIA income from highest to lowest?
================================== Ai Message ==================================

The order of NVIDIA's Net Income from highest to lowest is as follows: 
1. 2024-01-31: 29.76 Billion USD
2. 2022-01-31: 9.752 Billion USD
3. 2023-01-31: 4.368 Billion USD
4. 2021-01-31: 4.332 Billion USD.


In [20]:
result = app.invoke(
    {"input": "What company information is this?"},
    config=config,
)
print(result["answer"])

The information provided is about NVIDIA, a technology company known for its graphics processing units (GPUs) and other semiconductor products. NVIDIA is a prominent player in the gaming, data center, and professional visualization markets.


In [21]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

What is NVIDIA Net Income?
================================== Ai Message ==================================

NVIDIA's Net Income for the date 2024-01-31 was 29.76 Billion USD, for 2023-01-31 was 4.368 Billion USD, for 2022-01-31 was 9.752 Billion USD, and for 2021-01-31 was 4.332 Billion USD. The Net Income represents the company's total earnings after expenses such as taxes and operating costs have been deducted.
================================ Human Message =================================

Order NVIDIA income from highest to lowest?
================================== Ai Message ==================================

The order of NVIDIA's Net Income from highest to lowest is as follows: 
1. 2024-01-31: 29.76 Billion USD
2. 2022-01-31: 9.752 Billion USD
3. 2023-01-31: 4.368 Billion USD
4. 2021-01-31: 4.332 Billion USD.
================================ Human Message =================================

What 